In [11]:
import pygame
import random
import copy
import pprint

pygame 2.5.2 (SDL 2.28.3, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [12]:
NUM_OF_CARDS_EACH_COLOUR = 13
NUM_OF_COLOURS = 4
ALL_COLOURS = ("Red","Blue","Yellow","Green","Pink")
NOW_COLOURS = ALL_COLOURS[:NUM_OF_COLOURS]
NUM_OF_WILDCARDS = 2
NUM_OF_SAME_CARD = 2

Card

In [13]:
class Card:
    def __init__(self, colour, number):
        self.colour = colour
        self.number = number

    def __repr__(self):
        return f"Card({self.colour} {self.number})"
    
    @classmethod
    @property
    def wildcard(cls):
        cls.wildcard = Card("Joker", 30)

Deck


In [14]:
class Deck:
    def __init__(self):
        cards_pool = NUM_OF_SAME_CARD * [Card(colour, i) for colour in NOW_COLOURS for i in range(1, NUM_OF_CARDS_EACH_COLOUR+1)]
        cards_pool += NUM_OF_WILDCARDS * [Card.wildcard]
        random.shuffle(cards_pool)
        self.cards_pool = cards_pool

Player

In [15]:
class Player:
    def __init__(self, name) -> None:
        self.name = name
        self.hands = []
    
    def draw_cards(self, deck):
        for _ in range(14):
            card = deck.cards_pool.pop()
            self.hands.append(card)
    
    def draw_one_card(self, deck):
        card = deck.cards_pool.pop()
        self.hands.append(card)
        
    def __str__(self) -> str:
        return self.name
    
    def __repr__(self):
        return self.__str__()
    

Board

In [16]:
class Board:
    def __init__(self) -> None:
        self.board = []
    
    def __str__(self) -> str:
        return str(self.board)
    
    def __repr__(self) -> str:
        return self.__str__()
    
    def add_one_card_set(self, card_set):
        self.board.append(card_set)
        
    def add_all_card_sets(self, card_sets):
        self.board.extend(card_sets)
        
    def add_card_set_with_num(self, card_set, num):
        if num < len(self.board):
            self.board[num].extend(card_set)
            return True
        elif num == len(self.board):
            self.add_one_card_set(card_set)
            return True
        else:
            return False  

Game

In [17]:
import itertools

class Game:
    def __init__(self) -> None:
        self.deck = Deck()
        self.board = Board()
        
    def is_group(self, cards):
        if len(cards) < 3:
            return False
        
        regular_cards = [card for card in cards if card.number != 30]     
         
        if len(set([card.number for card in regular_cards])) != 1:
            return False
        else:
            return len(set([card.colour for card in regular_cards])) == len(regular_cards)
        
    def is_run(self, cards):
        if len(cards) < 3:
            return False

        regular_cards = [card for card in cards if card.number != 30] 
        wild_cards_count = len([card for card in cards if card.number == 30]) 

        if len(set([card.colour for card in regular_cards])) != 1:
            return False
        else:
            cards_number = [card.number for card in regular_cards]
            cards_number.sort()
            for i in range(1, len(cards_number)):
                if cards_number[i] != cards_number[i-1] + 1:
                    if wild_cards_count == 0:
                        return False
                    wild_cards_count -= 1
            return True 
    
    def is_valid(self, cards):
        return self.is_group(cards) or self.is_run(cards)              
    
    def is_first_move_valid(self, cards):
        if self.is_valid(cards):                
            total_points = sum(card.number for card in cards)
            if total_points >= 30:
                return True
        else:
            return False
        
    def find_combinations(self, cards):
        combinations = []
        cards.sort(key=lambda card: card.number)
        print(cards)
        for i in range(len(cards)):
            for j in range(i, len(cards)):
                cards_temp = cards[i:j]
                cards_left = cards[0:i] + cards[j:]
                if self.is_valid(cards_temp) and self.is_valid(cards_left):
                    combinations.append(cards_temp)
                    combinations.append(cards_left)
                    
                    # if self.find_combinations(cards_left):
                    #     combinations.append(cards_left)
        return combinations        
    
    
    def check_combinations(self, tiles):
        tiles.sort(key=lambda tile: (tile.number, tile.colour))
        print(tiles)
        for i in range(1, len(tiles) + 1):
            first_group = tiles[:i]
            if self.is_valid(first_group):
                remaining_tiles = tiles[i:]
                if not remaining_tiles or self.check_combinations(remaining_tiles):
                    return 1 + (self.check_combinations(remaining_tiles) if remaining_tiles else 0)
        return 0
    
    
    def check_all_combinations(self, tiles, combination=[], remaining=None):
        if remaining is None:
            remaining = tiles.copy()

        if not remaining:
            return all(self.is_valid(group) for group in combination)

        for i in range(len(remaining)):
            next_tile = remaining.pop(0)
            for group in combination:
                group.append(next_tile)
                if self.check_all_combinations(tiles, combination, remaining):
                    return True
                group.pop()

            combination.append([next_tile])
            if self.check_all_combinations(tiles, combination, remaining):
                return True
            combination.pop()
            remaining.append(next_tile)
        return False
    
    
    
    def find_all_combinations(self, tiles):
        valid_combinations = set()
        self._find_combinations_recursive(tiles, [], tiles.copy(), valid_combinations)
        
        return [list(map(self._convert_to_tile, comb)) for comb in valid_combinations]

    def _convert_from_hashable(self, hashable_combination):
        return [[Card(color, int(number)) for number, color in (tile_str.split() for tile_str in group)] for group in hashable_combination]

    def _convert_to_tile(self, group):
        return [Card(color, int(number)) for number, color in (tile_str.split() for tile_str in group)]
    
    def _convert_combination_to_hashable(self, combination):
        return tuple(tuple(f"{tile.number} {tile.colour}" for tile in sorted(group, key=lambda t: (t.number, t.colour))) for group in combination)

    def _find_combinations_recursive(self, tiles, current_combination, remaining, valid_combinations):
        if not remaining:
            if all(self.is_valid(group) for group in current_combination):
                valid_combinations.add(self._convert_combination_to_hashable(current_combination))
            return

        for i in range(len(remaining)):
            next_tile = remaining.pop(0)
            for group in current_combination:
                group.append(next_tile)
                self._find_combinations_recursive(tiles, current_combination, remaining, valid_combinations)
                group.pop()

            new_combination = current_combination + [[next_tile]]
            self._find_combinations_recursive(tiles, new_combination, remaining, valid_combinations)

            remaining.append(next_tile)
            
    def first_move(self, player, card_sets):
        if all(card in player.hands for card_set in card_sets for card in card_set):
            if all(self.is_first_move_valid(card_set) for card_set in card_sets):
                player.hands = [card for card in player.hands if card not in card_set for card_set in card_sets]
                self.board.add_all_card_sets(card_sets)
                print('move success')
            else: 
                print("move failed")
        else:
            print("move failed")
            
            
    # card sets, corresponding nums in board
    def regular_move(self, player, card_sets, sets_nums_in_board):
        if all(card in player.hands for card_set in card_sets for card in card_set) and len(card_sets) == len(sets_nums_in_board):
            if all(self.is_valid(self.board.board[i].extend(card_sets[i])) for i in range(len(card_sets))):
                for i in range(len(card_sets)):
                    self.board.add_card_set_with_num(card_sets[i], sets_nums_in_board[i])
                

Test

In [18]:
cards_1 = [Card('red',11), Card('g', 11), Card('b', 11)]
cards_2 = [Card('red',11), Card('red', 12), Card('red', 13)]
cards_3 = [Card('red',11), Card('red', 12), Card('Joker', 30)]
game1 = Game()
print(game1.is_group(cards_1))
print(game1.is_group(cards_2))
print(game1.is_run(cards_1))
print(game1.is_run(cards_2))
print(game1.is_first_move_valid(cards_3))
print(f"cards_3 is run: {game1.is_run(cards_3)}")

cards_4 = [ Card('red',11), Card('red', 12), Card('red', 13),\
            Card('red',11), Card('g', 11),   Card('b', 11), \
           ]
cards_5 = [Card('red',11), Card('g', 11),   Card('b', 11), \
           Card('red',11), Card('red', 12), Card('red', 13), \
           Card('red',11), Card('red', 12), Card('red', 13)
           ]
cards_6 = [ Card('g',11), Card('g', 12), Card('g', 13),
            Card('red',11), Card('g', 11),   Card('b', 11), 
           ]
# print(game1.check_combinations(cards_6))
# print(game1.find_all_combinations(cards_6))
all_valid_combinations = game1.find_all_combinations(cards_6)
print("all the valid combinations:")
for combination in all_valid_combinations:
    print(combination)

True
False
False
True
True
cards_3 is run: True
all the valid combinations:
[[Card(g 11), Card(g 12), Card(g 13)], [Card(b 11), Card(g 11), Card(red 11)]]
[[Card(b 11), Card(g 11), Card(red 11)], [Card(g 11), Card(g 12), Card(g 13)]]


In [53]:
game1.board

[]

In [56]:
game = Game()

deck1 = game.deck
print(len(deck1.cards_pool))
print(deck1.cards_pool)

p1 = Player('player1')
p2 = Player('player2')
p3 = Player('player3')
p4 = Player('player4')
players = [p1, p2, p3, p4]
print(players)      
for player in players:
    print(f"{player} hands:{player.hands}")
    player.draw_cards(deck1)
    print(f"{player} hands:{player.hands}")
    print(f"{player} holds cards number: {len(player.hands)}")
    
print(len(deck1.cards_pool))

106
[Card(Blue 4), Card(Blue 2), Card(Green 13), Card(Blue 9), Card(Blue 2), Card(Blue 1), Card(Red 7), Card(Blue 3), Card(Yellow 2), Card(Green 2), Card(Green 12), Card(Yellow 6), Card(Blue 9), Card(Red 13), Card(Red 9), Card(Red 5), Card(Yellow 12), Card(Blue 7), Card(Yellow 9), Card(Green 8), Card(Red 2), Card(Green 6), Card(Green 13), Card(Green 6), Card(Green 4), Card(Yellow 5), Card(Yellow 13), Card(Yellow 9), Card(Red 5), Card(Yellow 10), Card(Yellow 12), Card(Joker 30), Card(Red 3), Card(Joker 30), Card(Red 3), Card(Blue 1), Card(Red 8), Card(Yellow 3), Card(Red 7), Card(Red 2), Card(Blue 5), Card(Blue 6), Card(Blue 4), Card(Red 6), Card(Red 4), Card(Blue 11), Card(Blue 8), Card(Blue 3), Card(Red 9), Card(Red 4), Card(Red 10), Card(Yellow 10), Card(Yellow 11), Card(Yellow 7), Card(Green 1), Card(Green 2), Card(Yellow 4), Card(Blue 13), Card(Yellow 4), Card(Blue 12), Card(Green 11), Card(Red 12), Card(Yellow 8), Card(Green 8), Card(Yellow 6), Card(Yellow 5), Card(Green 10), Card

In [57]:
p1.hands.sort(key=lambda tile: (tile.number, tile.colour))
p1.hands

[Card(Red 1),
 Card(Yellow 1),
 Card(Yellow 3),
 Card(Green 5),
 Card(Blue 6),
 Card(Blue 7),
 Card(Green 7),
 Card(Yellow 8),
 Card(Blue 11),
 Card(Yellow 11),
 Card(Green 12),
 Card(Red 12),
 Card(Blue 13),
 Card(Red 13)]

In [46]:
game = Game()
game.first_move(p1, [[p1.hands[9], p1.hands[11], p1.hands[13]], ])
game.board

move success


[[Card(Yellow 10), Card(Yellow 11), Card(Joker 30)]]

In [47]:
p1.hands

[Card(Green 1),
 Card(Blue 2),
 Card(Green 2),
 Card(Red 3),
 Card(Green 5),
 Card(Blue 6),
 Card(Red 7),
 Card(Green 9),
 Card(Red 10),
 Card(Green 11),
 Card(Red 13)]

In [48]:
game.board

[[Card(Yellow 10), Card(Yellow 11), Card(Joker 30)]]